# PyTerrier ECIR 2021 Tutorial Notebook - Part 2

This notebook provides experiences to attendees for building transformer pipelines in [PyTerrier](https://github.com/terrier-org/pyterrier). All experiments are conducted using the [CORD19 corpus](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7251955/) and the [TREC Covid test collection](https://ir.nist.gov/covidSubmit/).

This notebook is divided in two, following the structure of the slides. In particular, in Part 2A you will experience: 
 - understand the PyTerrier data model;
 - understand the general notion of PyTerrier transformers and operators for combining transformers into new transformers;
 - undertake experiments on the TREC Covid test collection.

In Part 2B, you will experience:
 - construct and learn learning-to-rank pipelines;
 - evaluate and analyse learning-to-rank pipelines.


# Setup

We install PyTerrier, as well as the [LightGBM](https://lightgbm.readthedocs.io/en/latest/) and [FastRank](https://github.com/jjfiv/fastrank) learning-to-rank implementations, and we will initialise PyTerrier.

In [ ]:
!pip install --upgrade fastrank lightgbm 
!pip install python-terrier

In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init()

# Indexing

We're going to create a new index for Cord19. In this index, we use the `blocks=True` argument, so that we have position information in the index.
Indexing takes ~1 minute.

In [ ]:
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19_blocks'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

## Transformers & Operators

You'll have noted that BatchRetrieve has a `transform()` method that takes as input a dataframe, and returns another dataframe, which is somehow a *transformation* of the earlier dataframe (e.g., a retrieval transformation). In fact, `BatchRetrieve` is just one of many similar objects in PyTerrier, which we call [transformers](https://pyterrier.readthedocs.io/en/latest/transformer.html) (represented by the `TransformerBase` class).

Let's give a look at a `BatchRetrieve` transformer, starting with one for the TF_IDF weighting model.

In [ ]:
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")

# check tfidf is a transformer...
print(isinstance(tfidf, pt.transformer.TransformerBase))

In [ ]:
# this prints the type hierarchy of the TF_IDF class
tfidf.__class__.__mro__

The interesting capability of all transformers is that they can be combined using Python operators (this is called *operator overloading*).

Concretely, imagine that you want to chain transformers together – e.g. rank documents first by Tf then re-ranked the *exact same* documents by TF_IDF. We can do this using the `>>` operator – we call this *composition*, or "*then*".

In [ ]:
# this is our first retrieval transformer
# it transform a queries dataframe to a results dataframe
tf = pt.BatchRetrieve(index, wmodel="Tf")

tf( cord19.get_topics(variant='title').head(1) )

In [ ]:
# now let's define a pipeline 
pipeline = tf >> tfidf
print(isinstance(tfidf, pt.transformer.TransformerBase))

pipeline( cord19.get_topics(variant='title').head(1) )

There are a number of PyTerrier operators – there are more examples in the [PyTerrier documentation on operators](https://pyterrier.readthedocs.io/en/latest/operators.html)

## Practice Task – Pipeline Construction

Create a ranker that performs the follinwg:
 - obtains the top 10 highest scoring documents by term frequency (`wmodel="Tf"`)
 - obtains the top 10 highest scoring documents by TF.IDF (`wmodel="TF_IDF"`)
 - reranks only those documents found in BOTH of the previous retrieval settings using BM25.

How many documents are retrieved by this full pipeline for the query `"chemical"`.
> If you obtain the correct solution, the document with docno `"37771"` should have a score of $12.426309	$ for query `"chemical"`.

Hints:
 - choose careully your [PyTerrier operators](https://pyterrier.readthedocs.io/en/latest/operators.html)
 - you should not need to perform any Pandas dataframe operations

In [ ]:
# SOLUTION

pipe = (tfidf %10) & (tf %10)
pipe = pipe >>  pt.BatchRetrieve(index, wmodel="BM25")
pipe.search("chemical")

## Other transformers in the toolbox

Lets start creating more interesting retrieval pipelines. We'll define these, but firstly lets recap on the PyTerrier datamodel:
 - $Q$: a set of queries
 - $D$: a set of documents
 - $R$: a set of retrieved documents for a set of queries

Lets use three: 
 - `pt.BatchRetrieve(index, wmodel="BM25")` - input $Q$ or $R$ (retrieval or reranking), output $R$
 - `pt.rewrite.SDM()` (sequential dependence proximity model) - input $Q$, output $Q$. 
 - `pt.rewrite.Bo1QueryExpansion(index)` - input $R$, output $Q$.




In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
sdm = pt.rewrite.SDM()
qe = pt.rewrite.Bo1QueryExpansion(index)

## Practice Task – Experimenting with Pipelines

Conduct an [Experiment](https://pyterrier.readthedocs.io/en/latest/experiments.html) comparing sequential dependence model and Bo1 query expansion on TREC CORD19 with the BM25 baseline. You will need to construct appropriate pipelines, by consdering the input and output datatypes of the `bm25`, `sdm` and `qe`. 

Which approaches result in significant increases in NDCG and MAP? Is NDCG@10 also improved?

In [ ]:
topics = cord19.get_topics(variant='title')
qrels = cord19.get_qrels()

In [ ]:
# SOLUTION
sdm_pipe = sdm >> bm25
qe_pipe = bm25 >> qe >> bm25

In [ ]:
# SOLUTION
pt.Experiment(
    [bm25, sdm_pipe, qe_pipe],
    topics,
    qrels,
    names=["BM25", "BM25 + SDM", "BM25 + QE"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10"])

# Part 2B – Learning to Rank

In this part of the notebook, you will experience constructing, learning, evaluating and analysing learning to rank pipelines.


Firstly, lets split out topics into train, validation and test sets. TREC Covid only has 50 topics, which isnt a lot for learning(!). We'll split this 35 for training, 5 for validation and 15 for evaluation. We will also examine statistical significance, even if this is artificial for 15 topics.

We're only going to-rank the top 10 documents for each query - hopefully learning to rank can help to re-rank the top 10 documents to be more effective.

In [ ]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

## Feature Set

Lets define our feature set.  We're going to have a total of 7 features:

1.   the BM25 abstract score;
2.   sequential dependence model, scored by BM25;
3.   does the abstract contain 'coronavirus covid', scored by BM25;
4.   the BM25 score on the title (even though we didnt index it earlier!);
5.   was the paper released/published in 2020? Recent papers were more useful for this task;
6.   does the paper have a DOI, i.e. is it a formal publication?
7.   the coordinate match score for the query - i.e. how many query terms appear in the abstract.

Several of these feature require additional metadata `["title", "date", "doi"]`. Fortunately, the TREC Covid dataset allows us to obtain more metadata after indexing. We use `pt.text.get_text(cord19, ["title", "date", "doi"])` to retrieve these extra metadata columns.


In [ ]:
ltr_feats1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"])  >> (
    pt.transformer.IdentityTransformer()
    ** #sequential dependence
    sdm_pipe
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> bm25)
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='BM25') ) 
    ** #date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** #has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    ** #abstract coordinate match
    pt.BatchRetrieve(index, wmodel="CoordinateMatch")
)

# for reference, lets record the feature names here too
fnames=["BM25", "SDM", 'coronavirus covid', 'title', "2020", "hasDoi", "CoordinateMatch"]

Lets see the output for a particular query. We can see that we now have extra document metadata columns `["title", "date", "doi"]`, as well as the all-important `"features"` columns. This makes dataframe have type $R_f$. Indeed,  it is this column that we use for learning.

In [ ]:
ltr_feats1.search("coronovirus origin")

We can also look at the raw features values (in this case for the first ranked document). Note that the BM25 in the "score" column above is also the first value in the feature array (20.54), because we used an identity transformer.

In [ ]:
ltr_feats1.search("coronovirus origin").iloc[0]["features"]

## Learning 

In this part of the the notebook, we apply three different learning to rank techniques:

 - coordinate ascent from FastRank, a listwise linear technique
 - random forests from `scikit-learn`, a pointwise regression tree technique
 - LambdaMART from LightGBM, a listwise regression tree technique

In each case, we take our feature pipeline, `ltr_feats1`, and we compose it (`>>`) with the learned model. We use `pt.ltr.apply_learned_model()` which knows how to deal with different learners.

The full pipeline is then fitted (learned) using `.fit()`, specifying the training topics and qrels. Importantly, the preceeding stages of the pipeline (retrieval and feature calculation) are applied to the training topics in order to obtained the results, which are then passed to the learning to rank technique. LightGBM has early stopping enabled, which uses a validation topics set – similarly the validation topics are transformed into validation results.

Finally, `%time` is notebook "magic command" which displays how long learning takes for each technique. Learning for each technique takes < 30 seconds.

In [ ]:
import fastrank

train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, cord19.get_qrels())

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf)

%time rf_pipe.fit(train_topics, cord19.get_qrels())

In [ ]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

max_depth=5, #was unlimited
num_leaves=16, #was 31
min_gain_to_split=0.1, #was 0
min_sum_hessian_in_leaf=100,
min_data_in_leaf=2, #was 1

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, cord19.get_qrels(), valid_topics, cord19.get_qrels())

## Evaluation

Lets now compare our ranking pipelines on our 15 topics with the BM25 baseline. In all cases, we're ranking only 10 results per query, so MAP will be lower. 

We'll report mean response time (`"mrt"`) as well as MAP, NDCG and NDCG@10 measures.

In [ ]:
pt.Experiment(
    [bm25 % RANK_CUTOFF, ca_pipe, rf_pipe, lmart_x_pipe],
    test_topics,
    qrels, 
    names=["BM25",  "BM25 + CA(7f)", "BM25 + RF(7f)", "BM25 + LMart(7f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

Thats really interesting – all three learned models could improve NDCG@10 over BM25, but the coordinate ascent model improved the most (significantly so on all three metrics, but again on only 15 queries). Coordinate Ascent improved upto 11 queries.

## Analysis

Lets start to analyse our learned models. Two things we could do is either a feature ablation study, or to evaluate the performance of each feature independently. To to that, we compose the feature pipeline (`ltr_feats1`) with `pt.ltr.feature_to_score(i)` for some feature number $i$.

In [ ]:
pt.Experiment(
    [ltr_feats1 >> pt.ltr.feature_to_score(i) for i in range(len(fnames))],
    test_topics,
    qrels, 
    names=fnames,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "num_rel_ret"])

Interesting, we observe that the 'coronavirus covid' feature achieved NDCG@10 of 0.572172. On average, this outperforms some of the regression tree-based learners. That coordinate ascent could outperform the strongest feature gives some credence to the appropriateness of such a linear learner in  environments without lots of training data.

Next, we analyse the actual learned models. For the coordinate ascent model, we plot the feature weights (note the log-scale y-axis); while for the regression-tree based techniques, we report the feature importance.

In [ ]:
import matplotlib.pyplot as plt, numpy as np

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(10, 6))

ax0.bar(np.arange(len(fnames)), ca_pipe[1].model.to_dict()['Linear']['weights'])
ax0.set_xticks(np.arange(len(fnames)))
ax0.set_xticklabels(fnames, rotation=45, ha='right')
ax0.set_title("Coordinate Ascent\n Feature Weights")
ax0.set_yscale('log')

ax1.bar(np.arange(len(fnames)), rf.feature_importances_)
ax1.set_xticks(np.arange(len(fnames)))
ax1.set_xticklabels(fnames, rotation=45, ha='right')
ax1.set_title("Random Forests\n Feature Importance")

ax2.bar(np.arange(len(fnames)), lmart_l.feature_importances_)
ax2.set_xticks(np.arange(len(fnames)))
ax2.set_xticklabels(fnames, rotation=45, ha='right')
ax2.set_title("$\lambda$MART\n Feature Importance")

fig.show()


## Practice Task – Concatenation

Our learned model has low recall because only 10 documents are re-ranked. Lets make a small function, `append_baseline()`, that can append the BM25 baselines results to the output of the learned model. This is defined using [transformer operators](https://pyterrier.readthedocs.io/en/latest/operators.html) (`^` and `%`).

As an exercise, apply `append_baseline()` to each of the learned model pipelines defined above, and report the MAP and NDCG computed on all 1000 ranked results. 

Which of the learned models result in significantly improved MAP and NDCG?


In [ ]:
def append_baseline(system, baseline, max_results = 1000):
    return (system ^ baseline) % max_results

In [ ]:
# SOLUTION
pt.Experiment(
    [append_baseline(system, bm25) for system in [bm25 % RANK_CUTOFF, ca_pipe, rf_pipe, lmart_x_pipe]],
    test_topics,
    cord19.get_qrels(), 
    names=["BM25",  "BM25 + CA(7f)", "BM25 + RF(7f)", "BM25 + LMart(7f)"],
    baseline=0,
    eval_metrics=["num_ret", "map", "ndcg", "ndcg_cut_10", "mrt"])

#  That's all folks

If you arent coming back to Part 3 of the tutorial, please dont forget to complete our exit quiz: https://forms.office.com/r/2WbpLiQmWV